In [3]:
from google.colab import drive
drive.mount('/content/drive') 

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import os
os.chdir("/content/drive/My Drive/MLGroupProject/Dataset/")

In [5]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Start

In [8]:
matches=pd.read_csv('./kaggle_data/matches.csv')  #reading match data

In [9]:
matches.columns

Index(['id', 'season', 'city', 'date', 'team1', 'team2', 'toss_winner',
       'toss_decision', 'result', 'dl_applied', 'winner', 'win_by_runs',
       'win_by_wickets', 'player_of_match', 'venue', 'umpire1', 'umpire2',
       'umpire3'],
      dtype='object')

In [10]:
matches.shape

(756, 18)

In [11]:
matches = matches.drop(['umpire3', 'umpire1', 'umpire2', 'city'], axis=1)

In [12]:
deliveries = pd.read_csv('./kaggle_data/deliveries.csv') #reading ball - by - ball data

In [13]:
deliveries.shape

(179078, 21)

In [14]:
# deliveries.groupby('total_runs').count()['match_id'].plot(kind="bar", title="Runs scored per delivery")


In [15]:
matches_copy = matches.copy()

In [16]:
# matches = matches_copy.copy()

In [17]:
set(matches['team1'].to_list()) #creating set for team names.

{'Chennai Super Kings',
 'Deccan Chargers',
 'Delhi Capitals',
 'Delhi Daredevils',
 'Gujarat Lions',
 'Kings XI Punjab',
 'Kochi Tuskers Kerala',
 'Kolkata Knight Riders',
 'Mumbai Indians',
 'Pune Warriors',
 'Rajasthan Royals',
 'Rising Pune Supergiant',
 'Rising Pune Supergiants',
 'Royal Challengers Bangalore',
 'Sunrisers Hyderabad'}

label encoding done below.

In [18]:
#label enoding below
team_dict = {'Mumbai Indians':'0','Kolkata Knight Riders': '1','Royal Challengers Bangalore': '2','Deccan Chargers': '3','Chennai Super Kings': '4',
                 'Rajasthan Royals': '5','Delhi Daredevils': '6','Gujarat Lions': '7','Kings XI Punjab': '8',
                 'Sunrisers Hyderabad': '9','Rising Pune Supergiants':'10','Kochi Tuskers Kerala': '11','Pune Warriors': '12','Rising Pune Supergiant': '10','Delhi Capitals':'6'}

#mapping the team names to label encoded integers.
matches['team1'] = matches['team1'].map(team_dict)
print(matches.shape)
matches['team2'] = matches['team2'].map(team_dict)
print(matches.shape)

matches['toss_winner'] = matches['toss_winner'].map(team_dict)
print(matches.shape)


matches['winner'] = matches['winner'].map(team_dict)
print(matches.shape)

toss_decision = {'field': 0, 'bat': 1}
matches['toss_decision'] = matches['toss_decision'].map(toss_decision)
print(matches.shape)


(756, 14)
(756, 14)
(756, 14)
(756, 14)
(756, 14)


In [19]:
matches.isna().sum()
matches = matches.dropna()

print(matches.shape)



(752, 14)


In [20]:
matches['team1'] = pd.to_numeric(matches['team1'])
matches['team2'] = pd.to_numeric(matches['team2'])
matches['toss_winner'] = pd.to_numeric(matches['toss_winner'])
matches['toss_decision'] = pd.to_numeric(matches['toss_decision'])
matches['winner'] = pd.to_numeric(matches['winner'])

In [21]:
matches.shape

(752, 14)

In [22]:
matches.dtypes

id                  int64
season              int64
date               object
team1               int64
team2               int64
toss_winner         int64
toss_decision       int64
result             object
dl_applied          int64
winner              int64
win_by_runs         int64
win_by_wickets      int64
player_of_match    object
venue              object
dtype: object

In [23]:
matches['season'] = matches['season']-2008 

In [24]:
# matches.sample(5)

converting winner of match as team 0 or team 1 instead of encoded integers so that it becomes a binary classification task.

In [25]:
matches.loc[matches['team1'] == matches['winner'], 'label'] = 1
matches.loc[matches['team1'] != matches['winner'], 'label'] = 0

In [26]:
matches.shape

(752, 15)

In [27]:
data = matches[['team1', 'team2','toss_winner', 'toss_decision', 'dl_applied', 'season', 'label']]

In [28]:
# data = data.dropna()

In [29]:
data.shape

(752, 7)

Creating input data for the classification task.

In [30]:
X = data[['team1', 'team2','toss_winner', 'toss_decision', 'dl_applied', 'season']].to_numpy()

In [31]:
# matches[['id','team1', 'team2','toss_winner', 'toss_decision', 'dl_applied', 'season','winner','win_by_runs']].corr()

In [32]:
X.shape

(752, 6)

In [33]:
y = data['label'].to_numpy()

In [34]:
y.shape

(752,)

implementing logistic regression model below with kfold cross validation.

X_train, y_train are the training inputs and output respectively

similarly X_test,y_test are the testing outputs of each fold. 

We are also calculating the accuracy, precision, recall,f1 score of each fold.

The code below is without hyperparameter tuning and reports the average of all the folds.

###Logistic Regression

In [35]:
from sklearn.model_selection import train_test_split

In [36]:
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report, precision_recall_fscore_support
from sklearn.model_selection import GridSearchCV

In [108]:
kf = KFold(n_splits=5)
kf.get_n_splits(X)
acc_test = []
acc_train = [] 
precision = []
recall = []
f1score = []
for train_index, test_index in kf.split(X):
    # print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    clf = LogisticRegression()
    clf.fit(X_train, y_train)
    # gridlr = GridSearchCV(LogisticRegression(),{"penalty":["l1","l2"],"C":[0.1,1,10,100]})
    # gridlr.fit(X_train,y_train)
    # print("best LR params:",gridlr.best_params_)

    y_pred = clf.predict(X_test)
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    print(tn, fp, fn, tp)
    pr = tp / (tp+fp)
    rec = tp/ (tp+fn)
    f1 = 2*pr*rec/(pr+rec)
    
    precision.append(pr)
    recall.append(rec)
    f1score.append(f1)

    print("pr", tp / (tp+fp))
    print('rec', tp/ (tp+fn))
    acc_train.append(clf.score(X_train, y_train))
    acc_test.append(clf.score(X_test, y_test))
    # print(precision_recall_fscore_support(y_test, y_pred))
    # print(classification_report(y_test, y_pred))

64 11 64 12
pr 0.5217391304347826
rec 0.15789473684210525
59 37 41 14
pr 0.27450980392156865
rec 0.2545454545454545
62 14 63 11
pr 0.44
rec 0.14864864864864866
79 1 62 8
pr 0.8888888888888888
rec 0.11428571428571428
76 14 51 9
pr 0.391304347826087
rec 0.15


In [109]:
sum(acc_train) / len(acc_train) #training accuracy

0.5668232900868431

In [110]:
print("Logistic Regression: ")
print("Accuracy:",sum(acc_test) / len(acc_test),"Precision:",sum(precision)/len(precision),"Recall:",sum(recall)/len(recall),"F1 Score:",sum(f1score)/len(f1score))


Logistic Regression: 
Accuracy: 0.5240176600441501 Precision: 0.5032884342142654 Recall: 0.16507491086438456 F1 Score: 0.22963930469836588


We have implemented the decision tree classifier below using kfold cross validation. We have also calculated all the performance metrics as done above.

### Decision Tree

In [40]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [41]:
from sklearn.model_selection import KFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix, classification_report, precision_recall_fscore_support

In [111]:
kf = KFold(n_splits=5)
kf.get_n_splits(X)
acc_test = []
acc_train = [] 
precision = []
recall = []
f1score = []
for train_index, test_index in kf.split(X):
    # print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    clf = DecisionTreeClassifier()
    clf.fit(X_train, y_train)
  

    y_pred = clf.predict(X_test)
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    print(tn, fp, fn, tp)
    pr = tp / (tp+fp)
    rec = tp/ (tp+fn)
    f1 = 2*pr*rec/(pr+rec)
    
    precision.append(pr)
    recall.append(rec)
    f1score.append(f1)

    print("pr", tp / (tp+fp))
    print('rec', tp/ (tp+fn))
    acc_train.append(clf.score(X_train, y_train))
    acc_test.append(clf.score(X_test, y_test))
    # print(precision_recall_fscore_support(y_test, y_pred))
    # print(classification_report(y_test, y_pred))

39 36 42 34
pr 0.4857142857142857
rec 0.4473684210526316
43 53 26 29
pr 0.35365853658536583
rec 0.5272727272727272
44 32 44 30
pr 0.4838709677419355
rec 0.40540540540540543
49 31 43 27
pr 0.46551724137931033
rec 0.38571428571428573
50 40 33 27
pr 0.40298507462686567
rec 0.45


In [112]:
sum(acc_train) / len(acc_train) #training accuracy

0.9444817883814904

In [114]:
print("Decision Tree")
print("Accuracy:",sum(acc_test) / len(acc_test),"Precision:",sum(precision)/len(precision),"Recall:",sum(recall)/len(recall),"F1 Score:",sum(f1score)/len(f1score))

Decision Tree
Accuracy: 0.494719646799117 Precision: 0.43834922120955255 Recall: 0.44315216788900996 F1 Score: 0.4354718819746094


We have implemented the support vector machine classifier below using kfold cross validation. We have also calculated all the performance metrics as done above. This is again without hyperparameter tuning and reports the average of all the folds.

### SVM

In [115]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [116]:
from sklearn.model_selection import KFold
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, classification_report, precision_recall_fscore_support

In [117]:
kf = KFold(n_splits=5)
kf.get_n_splits(X)
acc_test = []
acc_train = [] 
precision = []
recall = []
f1score = []
for train_index, test_index in kf.split(X):
    # print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    clf = SVC()
    clf.fit(X_train, y_train)

    y_pred = clf.predict(X_test)
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    print(tn, fp, fn, tp)
    pr = tp / (tp+fp)
    rec = tp/ (tp+fn)
    f1 = 2*pr*rec/(pr+rec)
    
    precision.append(pr)
    recall.append(rec)
    f1score.append(f1)

    print("pr", tp / (tp+fp))
    print('rec', tp/ (tp+fn))
    acc_train.append(clf.score(X_train, y_train))
    acc_test.append(clf.score(X_test, y_test))
    # print(precision_recall_fscore_support(y_test, y_pred))
    # print(classification_report(y_test, y_pred))

61 14 63 13
pr 0.48148148148148145
rec 0.17105263157894737
58 38 38 17
pr 0.3090909090909091
rec 0.3090909090909091
73 3 67 7
pr 0.7
rec 0.0945945945945946
72 8 63 7
pr 0.4666666666666667
rec 0.1
80 10 56 4
pr 0.2857142857142857
rec 0.06666666666666667


In [118]:
sum(acc_train) / len(acc_train) #training accuracy

0.5847773091359363

In [119]:
print("SVM")
print("Accuracy:",sum(acc_test) / len(acc_test),"Precision:",sum(precision)/len(precision),"Recall:",sum(recall)/len(recall),"F1 Score:",sum(f1score)/len(f1score))

SVM
Accuracy: 0.5213509933774835 Precision: 0.44859066859066854 Recall: 0.14828096038622354 F1 Score: 0.2001997501369289


### Random Forest

We have implemented the Random Forest classifier below using kfold cross validation. We have also calculated all the performance metrics as done above. This is without hyperparameter tuning and reports the average of all the folds.

In [120]:
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report, precision_recall_fscore_support
from sklearn.model_selection import GridSearchCV

In [121]:
kf = KFold(n_splits=5)
kf.get_n_splits(X)
acc_test = []
acc_train = [] 
precision = []
recall = []
f1score = []
for train_index, test_index in kf.split(X):
    # print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    
  
    clf = RandomForestClassifier()
    clf.fit(X_train, y_train)
    y_pred = gridrf.predict(X_test)

    y_pred = clf.predict(X_test)
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    print(tn, fp, fn, tp)
    pr = tp / (tp+fp)
    rec = tp/ (tp+fn)
    f1 = 2*pr*rec/(pr+rec)
    
    precision.append(pr)
    recall.append(rec)
    f1score.append(f1)

    print("pr", tp / (tp+fp))
    print('rec', tp/ (tp+fn))
    acc_train.append(gridrf.score(X_train, y_train))
    acc_test.append(gridrf.score(X_test, y_test))
    # print(precision_recall_fscore_support(y_test, y_pred))
    # print(classification_report(y_test, y_pred))

43 32 45 31
pr 0.49206349206349204
rec 0.40789473684210525
52 44 26 29
pr 0.3972602739726027
rec 0.5272727272727272
46 30 46 28
pr 0.4827586206896552
rec 0.3783783783783784
45 35 44 26
pr 0.4262295081967213
rec 0.37142857142857144
53 37 36 24
pr 0.39344262295081966
rec 0.4


In [122]:
sum(acc_train) / len(acc_train) #training accuracy

0.5771294796601456

In [123]:
print("Random Forest")
print("Accuracy:",sum(acc_test) / len(acc_test),"Precision:",sum(precision)/len(precision),"Recall:",sum(recall)/len(recall),"F1 Score:",sum(f1score)/len(f1score))

Random Forest
Accuracy: 0.5771567328918322 Precision: 0.43835090357465817 Recall: 0.4169948827843564 F1 Score: 0.4234102738943159


### All Together

Below is a classification task done side by side using different models and hyperparameter tuning. We have also included MLPclassifier in this deadline which can also be found below. We shuffle the dataset to begin with and then find the best hyperparameters using gridsearch.

In [103]:
from sklearn.model_selection import train_test_split


from sklearn.utils import shuffle
X, y = shuffle(X, y, random_state=3)

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.2)
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPClassifier
clf = LogisticRegression(random_state=0)
clf.fit(X_train, y_train)
gridlr = GridSearchCV(LogisticRegression(),{"penalty":["l1","l2"],"C":[0.1,1,10,100]})
gridlr.fit(X_train,y_train)
print("best LR params:",gridlr.best_params_)

rf = RandomForestClassifier()
rf.fit(X_train,y_train)
gridrf = GridSearchCV(RandomForestClassifier(),{"n_estimators" :[1,10,100,1000],"criterion":["gini", "entropy"],"max_depth":[1,2,3,4,5,10]})
gridrf.fit(X_train,y_train)
print("Best RF params:",gridrf.best_params_)

dt = DecisionTreeClassifier()
dt.fit(X_train,y_train)
griddt = GridSearchCV(DecisionTreeClassifier(),{"criterion":["gini", "entropy"],"splitter":["best","random"],"max_depth":[1,2,3,4,5,10]})
griddt.fit(X_train,y_train)
print("Best DT params:",griddt.best_params_)

svcpred = SVC()
svcpred.fit(X_train,y_train)
gridsvc = GridSearchCV(SVC(),{"C":[0.1,1,10,100],"kernel":["linear", "poly", "rbf", ],"gamma":["scale","auto"]})
gridsvc.fit(X_train,y_train)
print("Best SVC params:",gridsvc.best_params_)

mlp = MLPClassifier(hidden_layer_sizes=(8,6,4,2, ))
gridmlp = GridSearchCV(MLPClassifier(),{"hidden_layer_sizes":[(8,6,4,2, ),(10,8,6,),(12,8,4,2),(8,8,4,2)],"activation":["relu","tanh","logistic"],"learning_rate":["constant","adaptive"],"max_iter":[500],"early_stopping":[True]})
gridmlp.fit(X_train,y_train)
print("Best MLP params:",gridmlp.best_params_)
# mlp.fit(X_train,y_train)


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: Solver lbfgs supports only 'l2' or 'none' penalties, got l1 penalty.

  FitFailedWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: Solver lbfgs supports only 'l2' or 'none' penalties, got l1 penalty.

  FitFailedWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: Solver lbfgs supports only 'l2' or 'none' penalties, got l1 penalty.

  FitFailedWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/model_se

best LR params: {'C': 0.1, 'penalty': 'l2'}
Best RF params: {'criterion': 'entropy', 'max_depth': 4, 'n_estimators': 100}
Best DT params: {'criterion': 'gini', 'max_depth': 4, 'splitter': 'random'}
Best SVC params: {'C': 1, 'gamma': 'scale', 'kernel': 'linear'}
Best MLP params: {'activation': 'tanh', 'early_stopping': True, 'hidden_layer_sizes': (8, 8, 4, 2), 'learning_rate': 'constant', 'max_iter': 500}


In [104]:
y_pred_logreg = gridlr.predict(X_test)
y_pred_svc = gridsvc.predict(X_test)
y_pred_rf = gridrf.predict(X_test)
y_pred_dt = griddt.predict(X_test)
y_pred_mlp = gridmlp.predict(X_test)

In [105]:
from sklearn.metrics import classification_report,accuracy_score

In [106]:
print("Logistic Regression: ")
print(classification_report(y_test, y_pred_logreg))

print("Decision Tree: ")
print(classification_report(y_test, y_pred_dt))

print("SVM: ")
print(classification_report(y_test, y_pred_svc))

print("Random Forest: ")
print(classification_report(y_test, y_pred_rf))
print("MLP: ")
print(classification_report(y_test, y_pred_mlp))

Logistic Regression: 
              precision    recall  f1-score   support

         0.0       0.63      0.81      0.71        95
         1.0       0.38      0.20      0.26        56

    accuracy                           0.58       151
   macro avg       0.51      0.50      0.48       151
weighted avg       0.54      0.58      0.54       151

Decision Tree: 
              precision    recall  f1-score   support

         0.0       0.64      0.76      0.69        95
         1.0       0.39      0.27      0.32        56

    accuracy                           0.58       151
   macro avg       0.52      0.51      0.51       151
weighted avg       0.55      0.58      0.55       151

SVM: 
              precision    recall  f1-score   support

         0.0       0.63      0.81      0.71        95
         1.0       0.36      0.18      0.24        56

    accuracy                           0.58       151
   macro avg       0.49      0.49      0.47       151
weighted avg       0.53      0

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
